In [13]:
import pandas as pd
import re
from urllib.parse import urlparse

In [33]:
# Read the Sales Data file
df_SalesInfo = pd.read_csv('/Users/daan/Desktop/HVA/Database management/salesfinal1.csv', sep=';', encoding='ISO-8859-1')
df_MovieTable = pd.read_csv('/Users/daan/Desktop/HVA/Database management/allMoviesWithUniqueID kopie.csv', sep=None, engine='python', encoding='ISO-8859-1')
df_MovieTable = df_MovieTable.rename(columns={"ID": "Movie_ID"})

# Check which columns exist in the dataframe
print("Columns in SalesInfo DataFrame:", df_SalesInfo.columns)

# Define the list of columns to drop
columns_to_drop = ['Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
                   'runtime', 'keywords', 'creative_type', 'url']

# Drop columns that are present in the dataframe
df_SalesInfo = df_SalesInfo.drop(columns=[col for col in columns_to_drop if col in df_SalesInfo.columns])

# Define a text cleaning function
def clean_text(text):
    if pd.isna(text):  # Check if the text is NaN
        return ''  # Return an empty string or handle NaN as needed
    # Convert text to lowercase and remove special characters
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the cleaning function to the 'title' column
df_SalesInfo['title'] = df_SalesInfo['title'].apply(clean_text)

# Normalize titles for both dataframes
df_SalesInfo['normalized_title'] = df_SalesInfo['title'].apply(clean_text)
df_MovieTable['normalized_title'] = df_MovieTable['TITLE'].apply(clean_text)

# Merge dataframes on the normalized title
df_SalesInfo = df_SalesInfo.merge(df_MovieTable[['normalized_title', 'Movie_ID']], 
                                  on='normalized_title', how='left')

# Convert the 'Movie_ID' column to integer to remove decimals
df_SalesInfo['Movie_ID'] = df_SalesInfo['Movie_ID'].fillna(0).astype(int)

# Remove duplicate rows based on all columns
df_SalesInfo = df_SalesInfo.drop_duplicates()

# Display the columns and last few rows
print(df_SalesInfo.columns)
print(df_SalesInfo.tail())



Columns in SalesInfo DataFrame: Index(['year', 'release_date', 'title', 'genre', 'international_box_office',
       'domestic_box_office', 'worldwide_box_office', 'production_budget',
       'Unnamed: 8', 'opening_weekend', 'theatre_count', 'avg run per theatre',
       'runtime', 'keywords', 'creative_type', 'url'],
      dtype='object')
Index(['year', 'release_date', 'title', 'genre', 'international_box_office',
       'domestic_box_office', 'worldwide_box_office', 'production_budget',
       'normalized_title', 'Movie_ID'],
      dtype='object')
       year    release_date title            genre international_box_office  \
31594  2021   November 18th                 Comedy                 13744,00   
31595  2019  September 27th                 Comedy               1975617,00   
31596  2019  September 27th                 Comedy               1975617,00   
31597  2020    January 16th        Romantic Comedy               8060952,00   
31598  2020    January 16th        Romantic Comedy

In [34]:
print(df_SalesInfo['title'].apply(type).value_counts())

title
<class 'str'>    31599
Name: count, dtype: int64


In [35]:
unmatched_Sales = df_SalesInfo[df_SalesInfo['Movie_ID'].isna()]

print("Unmatched rows in df_SalesInfo:")
print(unmatched_Sales[['title', 'normalized_title']])

#merged_df.to_csv('merged.csv', index=False, encoding='ISO-8859-1')

Unmatched rows in df_SalesInfo:
Empty DataFrame
Columns: [title, normalized_title]
Index: []


In [36]:
df_SalesInfo = df_SalesInfo.drop(columns=['domestic_box_office', 'worldwide_box_office', 'normalized_title'])

In [37]:
df_SalesInfo.to_csv('/Users/daan/Desktop/HVA/Database management/FinalSales1.csv', index=False)